In [48]:
import pandas as pd 

df = pd.read_csv('cleaned_sales.csv')
df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960,2,0.0,41.9136
1,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.620,2,0.0,6.8714
2,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.2,2.5164
3,6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.860,7,0.0,14.1694
4,7,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.280,4,0.0,1.9656


In [50]:
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='superstore_db',
    connection_timeout=600
)

cursor = connection.cursor()
print("✅ Connected to MySQL successfully!")

✅ Connected to MySQL successfully!


In [51]:
# 1️⃣ Insert Regions
regions = df['region'].drop_duplicates().reset_index(drop=True)


In [52]:
regions 

0      South
1       West
2    Central
3       East
Name: region, dtype: object

In [53]:

# 2️⃣ Insert Customers
customers = df[['customer_name', 'segment', 'country', 'city']].drop_duplicates().reset_index(drop=True)


In [54]:
customers

,customer_name,segment,country,city
0,Claire Gute,Consumer,United States,Henderson
1,Darrin Van Huff,Corporate,United States,Los Angeles
2,Sean O'Donnell,Consumer,United States,Fort Lauderdale
3,Brosina Hoffman,Consumer,United States,Los Angeles
4,Andrew Allen,Consumer,United States,Concord
...,...,...,...,...
4410,Maris LaWare,Consumer,United States,Los Angeles
4411,Ruben Ausman,Corporate,United States,Athens
4412,Tom Boeckenhauer,Consumer,United States,Miami
4413,Dave Brooks,Consumer,United States,Costa Mesa


In [55]:

# 3️⃣ Insert Products
products = df[['product_name', 'category', 'sub-category']].drop_duplicates().reset_index(drop=True)

In [56]:
products

,product_name,category,sub-category
0,Bush Somerset Collection Bookcase,Furniture,Bookcases
1,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels
2,Eldon Fold 'N Roll Cart System,Office Supplies,Storage
3,Eldon Expressions Wood and Plastic Desk Access...,Furniture,Furnishings
4,Newell 322,Office Supplies,Art
...,...,...,...
1749,Cisco 8961 IP Phone Charcoal,Technology,Machines
1750,Hon 2111 Invitation Series Corner Table,Furniture,Tables
1751,"Eureka Hand Vacuum, Bagless",Office Supplies,Appliances
1752,Chromcraft Bull-Nose Wood Oval Conference Tabl...,Furniture,Tables


In [57]:
for region in df['region']:
    cursor.execute("""
        INSERT IGNORE INTO Regions (RegionName)
        VALUES (%s)
    """, (region,))
connection.commit()
print("✅ Regions inserted successfully!")


✅ Regions inserted successfully!


In [58]:
for _, row in df.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO Customers (CustomerName, Segment, Country, City)
        VALUES (%s, %s, %s, %s)
    """, (row['customer_name'], row['segment'], row['country'], row['city']))
connection.commit()
print("✅ Customers inserted successfully!")


✅ Customers inserted successfully!


In [60]:
for _, row in df.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO Products (ProductName, Category, SubCategory)
        VALUES (%s, %s, %s)
    """, (row['product_name'], row['category'], row['sub-category']))
connection.commit()
print("✅ Products inserted successfully!")


✅ Products inserted successfully!


In [61]:
# Create lookup dictionaries for foreign keys
cursor.execute("SELECT RegionID, RegionName FROM Regions")
region_dict = dict(cursor.fetchall())

cursor.execute("SELECT CustomerID, CustomerName FROM Customers")
customer_dict = dict(cursor.fetchall())

cursor.execute("SELECT ProductID, ProductName FROM Products")
product_dict = dict(cursor.fetchall())

print("✅ Foreign key mappings ready!")


✅ Foreign key mappings ready!


In [33]:
for _, row in df.iterrows():
    try:
        order_date = datetime.strptime(str(row['order_date']), "%d/%m/%Y").date()
        ship_date = datetime.strptime(str(row['ship_date']), "%d/%m/%Y").date()
    except:
        continue  # skip if date format invalid

    cursor.execute("""
        INSERT IGNORE INTO Orders (OrderDate, ShipDate, ShipMode, CustomerID, RegionID)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        order_date,
        ship_date,
        row['ship_mode'],
        customer_dict.get(row['customer_name']),
        region_dict.get(row['region'])
    ))
connection.commit()
print("✅ Orders inserted successfully!")


✅ Orders inserted successfully!


In [62]:
# Create mapping for OrderID (MySQL auto increments, so fetch again)
cursor.execute("SELECT OrderID, CustomerID FROM Orders")
order_map = dict(cursor.fetchall())

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO OrderDetails (OrderID, ProductID, Quantity, Discount, Sales, Profit)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        order_map.get(customer_dict.get(row['customer_name'])),  # based on CustomerID mapping
        product_dict.get(row['product_name']),
        row['quantity'],
        row['discount'],
        row['sales'],
        row['profit']
    ))
connection.commit()
print("✅ OrderDetails inserted successfully!")


✅ OrderDetails inserted successfully!
